# Test Cases for Human-in-the-Loop


Let' see this working in practice by importing the email assistant with HITL from our module, `src/email_assistant/email_assistant_hitl.py`. We'll compile the graph with a [checkpointer](https://langchain-ai.github.io/langgraph/concepts/memory/#short-term-memory), which allows us to persist the state of the graph after interruption. We can then resume execution from the same state after the human has responded.

> **Note:** here we change to the parent directory (`%cd ..`) to access our project's module structure, which contains reusable prompts and components. The autoreload extensions ensure any changes to these modules are automatically reflected in the notebook without requiring kernel restarts. This allows us to organize our prompts in a dedicated module rather than defining them inline, making them easier to maintain and reuse across the notebooks! You can see all these files in: `src/email_assistant`

In [1]:
%cd ..
%load_ext autoreload
%autoreload 2

import uuid
from langgraph.checkpoint.memory import MemorySaver
from email_assistant.email_assistant_hitl import overall_workflow

/Users/rlm/Desktop/Code/interrupt_workshop


## Accept `write_email` and `schedule_meeting`

This test demonstrates the fundamental HITL approval flow when a user accepts all agent actions:
1. An email about tax planning is received and classified as "RESPOND"
2. The agent checks calendar availability for both suggested dates (Tuesday and Thursday)
3. The agent proposes scheduling a meeting on Tuesday at 2:00 PM for 45 minutes
4. The user reviews and ACCEPTS the meeting request without changes
5. The agent drafts a confirmation email to send to the client
6. The user reviews and ACCEPTS the email draft without changes
7. The agent marks the workflow as complete

This scenario shows how human-in-the-loop works at its most basic level - humans provide oversight while the agent handles the execution.

In [36]:
import uuid

# Respond - Meeting Request Email
email_input_respond = {
    "to": "Lance Martin <lance@company.com>",
    "author": "Project Manager <pm@client.com>",
    "subject": "Tax season let's schedule call",
    "email_thread": "Lance,\n\nIt's tax season again, and I wanted to schedule a call to discuss your tax planning strategies for this year. I have some suggestions that could potentially save you money.\n\nAre you available sometime next week? Tuesday or Thursday afternoon would work best for me, for about 45 minutes.\n\nRegards,\nProject Manager"
}

# Compile the graph
checkpointer = MemorySaver()
graph = overall_workflow.compile(checkpointer=checkpointer)
thread_id_1 = uuid.uuid4()
thread_config_1 = {"configurable": {"thread_id": thread_id_1}}

# Run the graph until the first interrupt 
# Email will be classified as "respond" 
# Agent will create a schedule_meeting and write_email tool call
print("Running the graph until the first interrupt...")
for chunk in graph.stream({"email_input": email_input_respond}, config=thread_config_1):
    # Inspect interrupt object if present
    if '__interrupt__' in chunk:
        Interrupt_Object = chunk['__interrupt__'][0]
        print("\nINTERRUPT OBJECT:")
        print(f"Action Request: {Interrupt_Object.value[0]['action_request']}")

Running the graph until the first interrupt...
📧 Classification: RESPOND - This email requires a response

INTERRUPT OBJECT:
Action Request: {'action': 'schedule_meeting', 'args': {'attendees': ['pm@client.com', 'lance@company.com'], 'subject': 'Tax Planning Strategies Discussion', 'duration_minutes': 45, 'preferred_day': '2025-05-06', 'start_time': 14}}


Accept the `schedule_meeting` tool call

After the agent proposes scheduling a meeting, it creates an interrupt to seek human approval. In this step, we simulate a user accepting the proposed meeting parameters without changes. This allows the workflow to proceed to the next step (drafting a confirmation email). The interrupt object contains critical information:

1. The action type (`schedule_meeting`)
2. The proposed meeting parameters (attendees, subject, duration, day, time)
3. Configuration options for what kinds of responses are allowed

When the user accepts, the tool is executed as proposed without modification.

In [37]:
from langgraph.types import Command

print(f"\nSimulating user accepting the {Interrupt_Object.value[0]['action_request']} tool call...")
for chunk in graph.stream(Command(resume=[{"type": "accept"}]), config=thread_config_1):
    # Inspect interrupt object if present
    if '__interrupt__' in chunk:
        Interrupt_Object = chunk['__interrupt__'][0]
        print("\nINTERRUPT OBJECT:")
        print(f"Action Request: {Interrupt_Object.value[0]['action_request']}")


Simulating user accepting the {'action': 'schedule_meeting', 'args': {'attendees': ['pm@client.com', 'lance@company.com'], 'subject': 'Tax Planning Strategies Discussion', 'duration_minutes': 45, 'preferred_day': '2025-05-06', 'start_time': 14}} tool call...

INTERRUPT OBJECT:
Action Request: {'action': 'write_email', 'args': {'to': 'pm@client.com', 'subject': "Re: Tax season let's schedule call", 'content': 'Hello,\n\nThank you for reaching out. I am available on Tuesday, May 6th at 2:00 PM for a 45-minute call to discuss tax planning strategies. I have scheduled the meeting accordingly.\n\nLooking forward to your suggestions.\n\nBest regards,\nLance'}}


Accept the `write_email` tool call

With the meeting scheduled, the agent now drafts a confirmation email to the client. The interrupt contains:

1. The email tool action with recipient, subject, and proposed content
2. The formatted email showing what will be sent
3. The same configuration options for user response types

When the user accepts, the email is sent as written. After this step, the agent marks the task as complete with the `Done` tool call, and the workflow ends. The complete message history shows all the steps taken:

1. Initial email processing
2. Calendar availability checks for both days
3. Meeting scheduling with confirmation
4. Email composition and sending
5. Workflow completion

In [13]:
print(f"\nSimulating user accepting the {Interrupt_Object.value[0]['action_request']} tool call...")
for chunk in graph.stream(Command(resume=[{"type": "accept"}]), config=thread_config_1):
    # Inspect response_agent most recent message
    if 'response_agent' in chunk:
        chunk['response_agent']['messages'][-1].pretty_print()
    # Inspect interrupt object if present
    if '__interrupt__' in chunk:
        Interrupt_Object = chunk['__interrupt__'][0]
        print("\nINTERRUPT OBJECT:")
        print(f"Action Request: {Interrupt_Object.value[0]['action_request']}")


Simulating user accepting the {'action': 'write_email', 'args': {'to': 'pm@client.com', 'subject': "Re: Tax season let's schedule call", 'content': 'Hello,\n\nThank you for reaching out. I have scheduled a call for us to discuss tax planning strategies on Tuesday, April 22, 2025, at 2:00 PM. The meeting is set for 45 minutes.\n\nLooking forward to our discussion.\n\nBest regards,\n\nLance Martin'}} tool call...
================================== Ai Message ==================================
Tool Calls:
  Done (call_h6a55b78txHk7GV7yQa9sQU9)
 Call ID: call_h6a55b78txHk7GV7yQa9sQU9
  Args:
    done: True


Look at the full message history, and see trace:

https://smith.langchain.com/public/82277f96-3abd-48e0-a4db-413f7572240d/r

In [14]:
state = graph.get_state(thread_config_1)
for m in state.values['messages']:
    m.pretty_print()

================================ Human Message =================================

Respond to the email: 

**Subject**: Tax season let's schedule call
**From**: Project Manager <pm@client.com>
**To**: Lance Martin <lance@company.com>

Lance,

It's tax season again, and I wanted to schedule a call to discuss your tax planning strategies for this year. I have some suggestions that could potentially save you money.

Are you available sometime next week? Tuesday or Thursday afternoon would work best for me, for about 45 minutes.

Regards,
Project Manager

---

================================== Ai Message ==================================
Tool Calls:
  check_calendar_availability (call_NrkaML1VPyeDXFNiQZUvfGIo)
 Call ID: call_NrkaML1VPyeDXFNiQZUvfGIo
  Args:
    day: 2025-04-22
  check_calendar_availability (call_cjRWdZCRnPsYsagEGZxASZht)
 Call ID: call_cjRWdZCRnPsYsagEGZxASZht
  Args:
    day: 2025-04-24
================================= Tool Message =================================

Ava

## Edit `write_email` and `schedule_meeting`

This test demonstrates how human modification works in the HITL flow:
1. We start with the same tax planning email as before
2. The agent proposes a meeting with the same parameters
3. This time, the user EDITS the meeting proposal to change:
   - Duration from 45 to 30 minutes
   - Meeting subject is made more concise
4. The agent adapts to these changes when drafting the email
5. The user further EDITS the email to be shorter and less formal
6. The workflow completes with both modifications incorporated

This scenario showcases one of the most powerful aspects of HITL: users can make precise modifications to agent actions before they are executed, ensuring the final outcome matches their preferences without having to handle all the details themselves.

In [14]:
# Same email as before
email_input_respond = {
    "to": "Lance Martin <lance@company.com>",
    "author": "Project Manager <pm@client.com>",
    "subject": "Tax season let's schedule call",
    "email_thread": "Lance,\n\nIt's tax season again, and I wanted to schedule a call to discuss your tax planning strategies for this year. I have some suggestions that could potentially save you money.\n\nAre you available sometime next week? Tuesday or Thursday afternoon would work best for me, for about 45 minutes.\n\nRegards,\nProject Manager"
}

# Compile the graph with new thread
checkpointer = MemorySaver()
graph = overall_workflow.compile(checkpointer=checkpointer)
thread_id_2 = uuid.uuid4()
thread_config_2 = {"configurable": {"thread_id": thread_id_2}}

# Run the graph until the first interrupt - will be classified as "respond" and the agent will create a write_email tool call
print("Running the graph until the first interrupt...")
for chunk in graph.stream({"email_input": email_input_respond}, config=thread_config_2):
    # Inspect interrupt object if present
    if '__interrupt__' in chunk:
        Interrupt_Object = chunk['__interrupt__'][0]
        print("\nINTERRUPT OBJECT:")
        print(f"Action Request: {Interrupt_Object.value[0]['action_request']}")

Running the graph until the first interrupt...
📧 Classification: RESPOND - This email requires a response

INTERRUPT OBJECT:
Action Request: {'action': 'schedule_meeting', 'args': {'attendees': ['pm@client.com', 'lance@company.com'], 'subject': 'Tax Planning Strategies Discussion', 'duration_minutes': 45, 'preferred_day': '2025-05-06', 'start_time': 14}}


Edit the `schedule_meeting` tool call

When the agent proposes the initial meeting schedule, we now simulate the user making modifications through the edit functionality. This demonstrates how the `edit` response type works:

1. The user receives the same meeting proposal as in the previous test
2. Instead of accepting, they modify the parameters:
   - Reducing duration from 45 to 30 minutes
   - Keeping the same day and time
3. The `edit` response includes the complete set of modified arguments
4. The interrupt handler replaces the original tool arguments with these edited ones
5. The tool is executed with the user's modifications

This shows how edit capability gives users precise control over agent actions while still letting the agent handle the execution details.

In [15]:
# Now simulate user editing the schedule_meeting tool call
print("\nSimulating user editing the schedule_meeting tool call...")
edited_schedule_args = {
    "attendees": ["pm@client.com", "lance@company.com"],
    "subject": "Tax Planning Discussion",
    "duration_minutes": 30,  # Changed from 45 to 30
    "preferred_day": "2025-05-06",
    "start_time": 14 
}
for chunk in graph.stream(Command(resume=[{"type": "edit", "args": {"args": edited_schedule_args}}]), config=thread_config_2):
    # Inspect response_agent most recent message
    if 'response_agent' in chunk:
        chunk['response_agent']['messages'][-1].pretty_print()
    # Inspect interrupt object if present
    if '__interrupt__' in chunk:
        Interrupt_Object = chunk['__interrupt__'][0]
        print("\nINTERRUPT OBJECT:")
        print(f"Action Request: {Interrupt_Object.value[0]['action_request']}")


Simulating user editing the schedule_meeting tool call...

INTERRUPT OBJECT:
Action Request: {'action': 'write_email', 'args': {'to': 'pm@client.com', 'subject': "Re: Tax season let's schedule call", 'content': 'Hello,\n\nThank you for reaching out regarding tax planning strategies. I am available on Tuesday, May 6th at 2:00 PM, and have scheduled a 30-minute call for us to discuss your suggestions. If you feel we need additional time, please let me know and I can adjust the meeting duration accordingly.\n\nLooking forward to our discussion.\n\nBest regards,\nLance'}}


Edit the `write_email` tool call

After accepting the modified meeting schedule, the agent drafts an email reflecting the 30-minute duration. Now we demonstrate how editing works with email content:

1. The agent has adapted its email to mention the shorter 30-minute duration
2. We simulate the user wanting an even more significant change to the email:
   - Completely rewriting the content to be shorter and less formal
   - Changing the meeting day mentioned in the email (showing how users can correct agent mistakes)
   - Requesting confirmation rather than stating the meeting as definite
3. The `edit` response contains the complete new email content
4. The tool arguments are updated with this edited content
5. The email is sent with the user's preferred wording

This example shows the power of HITL for complex communication tasks - the agent handles the structure and initial content, while humans can refine tone, style, and substance.

In [16]:
# Now simulate user editing the write_email tool call
print("\nSimulating user editing the write_email tool call...")
edited_email_args = {
    "to": "pm@client.com",
    "subject": "Re: Tax season let's schedule call",
    "content": "Hello Project Manager,\n\nThank you for reaching out about tax planning. I scheduled a 30-minute call next Thursday at 3:00 PM. Would that work for you?\n\nBest regards,\nLance Martin"
}
for chunk in graph.stream(Command(resume=[{"type": "edit", "args": {"args": edited_email_args}}]), config=thread_config_2):
    # Inspect response_agent most recent message
    if 'response_agent' in chunk:
        chunk['response_agent']['messages'][-1].pretty_print()
    # Inspect interrupt object if present
    if '__interrupt__' in chunk:
        Interrupt_Object = chunk['__interrupt__'][0]
        print("\nINTERRUPT OBJECT:")
        print(f"Action Request: {Interrupt_Object.value[0]['action_request']}")


Simulating user editing the write_email tool call...
================================== Ai Message ==================================
Tool Calls:
  Done (call_PVtmxQJgG0wdFh299b70ab1z)
 Call ID: call_PVtmxQJgG0wdFh299b70ab1z
  Args:
    done: True


Look at the full message history, and see trace, to view the edited tool calls:

https://smith.langchain.com/public/21769510-d57a-41e4-b5c7-0ddb23c237d8/r

In [17]:
state = graph.get_state(thread_config_2)
for m in state.values['messages']:
    m.pretty_print()

================================ Human Message =================================

Respond to the email: 

**Subject**: Tax season let's schedule call
**From**: Project Manager <pm@client.com>
**To**: Lance Martin <lance@company.com>

Lance,

It's tax season again, and I wanted to schedule a call to discuss your tax planning strategies for this year. I have some suggestions that could potentially save you money.

Are you available sometime next week? Tuesday or Thursday afternoon would work best for me, for about 45 minutes.

Regards,
Project Manager

---

================================== Ai Message ==================================
Tool Calls:
  check_calendar_availability (call_hKFyYkfY3fhQqflCAh2dm69E)
 Call ID: call_hKFyYkfY3fhQqflCAh2dm69E
  Args:
    day: 2025-05-06
  check_calendar_availability (call_xdLGVBhQ3ki1vOSzReNxCZYk)
 Call ID: call_xdLGVBhQ3ki1vOSzReNxCZYk
  Args:
    day: 2025-05-08
================================= Tool Message =================================

Ava

## Ignore `write_email`, `schedule_meeting`, and `question`

This test set demonstrates the "ignore" capability of the HITL system, showing how users can reject agent actions entirely:

1. First, we test ignoring a `schedule_meeting` request:
   - When the agent proposes scheduling a meeting, the user rejects it completely
   - The workflow ends immediately without scheduling anything
   
2. Second, we test accepting a meeting but ignoring the follow-up email:
   - The user accepts the meeting schedule
   - But when the agent drafts a confirmation email, the user ignores it
   - The meeting is still scheduled, but no email is sent
   - The workflow ends after the rejection

3. Third, we test ignoring a `question` tool call:
   - For a different email about brunch plans
   - The agent asks a clarifying question
   - The user ignores the question
   - The workflow ends without further action

The "ignore" capability is crucial for HITL systems as it allows users to stop potentially unwanted actions before they occur, providing an important safety mechanism and ensuring users maintain full control.

In [20]:
# Respond - Meeting Request Email
email_input_respond = {
    "to": "Lance Martin <lance@company.com>",
    "author": "Project Manager <pm@client.com>",
    "subject": "Tax season let's schedule call",
    "email_thread": "Lance,\n\nIt's tax season again, and I wanted to schedule a call to discuss your tax planning strategies for this year. I have some suggestions that could potentially save you money.\n\nAre you available sometime next week? Tuesday or Thursday afternoon would work best for me, for about 45 minutes.\n\nRegards,\nProject Manager"
}

# Compile the graph
checkpointer = MemorySaver()
graph = overall_workflow.compile(checkpointer=checkpointer)
thread_id_3 = uuid.uuid4()
thread_config_3 = {"configurable": {"thread_id": thread_id_3}}

# Run the graph until the first interrupt 
# Email will be classified as "respond" 
# Agent will create a schedule_meeting and write_email tool call
print("Running the graph until the first interrupt...")
for chunk in graph.stream({"email_input": email_input_respond}, config=thread_config_3):
    # Inspect interrupt object if present
    if '__interrupt__' in chunk:
        Interrupt_Object = chunk['__interrupt__'][0]
        print("\nINTERRUPT OBJECT:")
        print(f"Action Request: {Interrupt_Object.value[0]['action_request']}")

Running the graph until the first interrupt...
📧 Classification: RESPOND - This email requires a response

INTERRUPT OBJECT:
Action Request: {'action': 'schedule_meeting', 'args': {'attendees': ['pm@client.com', 'lance@company.com'], 'subject': 'Tax Planning Strategies Discussion', 'duration_minutes': 45, 'preferred_day': '2025-04-22', 'start_time': 14}}


Ignore the `schedule_meeting` tool call

In this test, we demonstrate rejection of the meeting scheduling proposal:

1. The agent suggests a 45-minute meeting on Tuesday at 2:00 PM
2. We simulate the user selecting "ignore" in the Agent Inbox interface
3. The `interrupt_handler` processes this rejection with special logic:
   - It adds a message explaining the user's choice to ignore
   - It returns a command to end the workflow
   - No meeting is scheduled and no email is sent

This capability is crucial for several reasons:
- Prevents incorrect actions from being executed
- Gives users veto power over any agent decision
- Provides a clear exit path when the agent's suggestion isn't appropriate

The trace shows how the workflow ends immediately after the ignore action.

In [21]:
print(f"\nSimulating user ignoring the {Interrupt_Object.value[0]['action_request']} tool call...")
for chunk in graph.stream(Command(resume=[{"type": "ignore"}]), config=thread_config_3):
    # Inspect interrupt object if present
    if '__interrupt__' in chunk:
        Interrupt_Object = chunk['__interrupt__'][0]
        print("\nINTERRUPT OBJECT:")
        print(f"Action Request: {Interrupt_Object.value[0]['action_request']}")


Simulating user ignoring the {'action': 'schedule_meeting', 'args': {'attendees': ['pm@client.com', 'lance@company.com'], 'subject': 'Tax Planning Strategies Discussion', 'duration_minutes': 45, 'preferred_day': '2025-04-22', 'start_time': 14}} tool call...


As you can see from the trace, we end:

https://smith.langchain.com/public/4e322b99-08ea-4d23-9653-475415ff3e33/r

Now, let's run again, but ignore the `write_email` tool call:

In [22]:
# Respond - Meeting Request Email
email_input_respond = {
    "to": "Lance Martin <lance@company.com>",
    "author": "Project Manager <pm@client.com>",
    "subject": "Tax season let's schedule call",
    "email_thread": "Lance,\n\nIt's tax season again, and I wanted to schedule a call to discuss your tax planning strategies for this year. I have some suggestions that could potentially save you money.\n\nAre you available sometime next week? Tuesday or Thursday afternoon would work best for me, for about 45 minutes.\n\nRegards,\nProject Manager"
}

# Compile the graph
checkpointer = MemorySaver()
graph = overall_workflow.compile(checkpointer=checkpointer)
thread_id_3 = uuid.uuid4()
thread_config_3 = {"configurable": {"thread_id": thread_id_3}}

# Run the graph until the first interrupt 
# Email will be classified as "respond" 
# Agent will create a schedule_meeting and write_email tool call
print("Running the graph until the first interrupt...")
for chunk in graph.stream({"email_input": email_input_respond}, config=thread_config_3):
    # Inspect interrupt object if present
    if '__interrupt__' in chunk:
        Interrupt_Object = chunk['__interrupt__'][0]
        print("\nINTERRUPT OBJECT:")
        print(f"Action Request: {Interrupt_Object.value[0]['action_request']}")

print(f"\nSimulating user accepting the {Interrupt_Object.value[0]['action_request']['action']} tool call...")
for chunk in graph.stream(Command(resume=[{"type": "accept"}]), config=thread_config_3):
    # Inspect response_agent most recent message
    if 'response_agent' in chunk:
        chunk['response_agent']['messages'][-1].pretty_print()
    # Inspect interrupt object if present
    if '__interrupt__' in chunk:
        Interrupt_Object = chunk['__interrupt__'][0]
        print("\nINTERRUPT OBJECT:")
        print(f"Action Request: {Interrupt_Object.value[0]['action_request']}")

print(f"\nSimulating user ignoring the {Interrupt_Object.value[0]['action_request']['action']} tool call...")
for chunk in graph.stream(Command(resume=[{"type": "ignore"}]), config=thread_config_3):
    # Inspect response_agent most recent message
    if 'response_agent' in chunk:
        chunk['response_agent']['messages'][-1].pretty_print()
    # Inspect interrupt object if present
    if '__interrupt__' in chunk:
        Interrupt_Object = chunk['__interrupt__'][0]
        print("\nINTERRUPT OBJECT:")
        print(f"Action Request: {Interrupt_Object.value[0]['action_request']}")

Running the graph until the first interrupt...
📧 Classification: RESPOND - This email requires a response

INTERRUPT OBJECT:
Action Request: {'action': 'schedule_meeting', 'args': {'attendees': ['pm@client.com', 'lance@company.com'], 'subject': 'Tax Planning Strategies Discussion', 'duration_minutes': 45, 'preferred_day': '2025-04-22', 'start_time': 14}}

Simulating user accepting the schedule_meeting tool call...

INTERRUPT OBJECT:
Action Request: {'action': 'write_email', 'args': {'to': 'pm@client.com', 'subject': "Re: Tax season let's schedule call", 'content': 'Hi,\n\nThank you for reaching out. I have scheduled our call to discuss tax planning strategies for Tuesday, April 22, 2025, at 2:00 PM. We will have 45 minutes to go over your suggestions.\n\nLooking forward to our discussion.\n\nBest regards,\n\nLance Martin'}}

Simulating user ignoring the write_email tool call...
================================= Tool Message =================================

User ignored this email dra

Again, we end:

https://smith.langchain.com/public/819be555-4919-4d14-bdd9-eb6f73a3bafe/r

In [23]:
state = graph.get_state(thread_config_3)
for m in state.values['messages']:
    m.pretty_print()

================================ Human Message =================================

Respond to the email: 

**Subject**: Tax season let's schedule call
**From**: Project Manager <pm@client.com>
**To**: Lance Martin <lance@company.com>

Lance,

It's tax season again, and I wanted to schedule a call to discuss your tax planning strategies for this year. I have some suggestions that could potentially save you money.

Are you available sometime next week? Tuesday or Thursday afternoon would work best for me, for about 45 minutes.

Regards,
Project Manager

---

================================== Ai Message ==================================
Tool Calls:
  check_calendar_availability (call_JpQevSvNkmodiACKRfONrqan)
 Call ID: call_JpQevSvNkmodiACKRfONrqan
  Args:
    day: 2025-04-22
  check_calendar_availability (call_3gJfzFvJkgY9lRVTpUF1D9Zt)
 Call ID: call_3gJfzFvJkgY9lRVTpUF1D9Zt
  Args:
    day: 2025-04-24
================================= Tool Message =================================

Ava

Now let's try an email that calls the `Question` tool

The `Question` tool represents another important HITL interaction pattern - the agent asking for additional information rather than taking immediate action. This test shows:

1. A different email scenario about a potential brunch invitation
2. The agent doesn't have enough information to respond definitively
3. Instead of guessing, it uses the `Question` tool to ask for clarification
4. We'll simulate the user ignoring this question

This demonstrates how the HITL system gracefully handles requests for information, and what happens when users choose not to engage with these requests.

In [24]:
# Respond - Meeting Request Email
email_input_respond = {
    "to": "Lance Martin <lance@company.com>",
    "author": "Partner <partner@home.com>",
    "subject": "Meet Jim and Lisa for brunch in 3 weeks?",
    "email_thread": "Hey, should we invite Jim and Lisa to brunch in 3 weeks? We could go to the new place on 17th that everyone is talking about."
}

# Compile the graph
checkpointer = MemorySaver()
graph = overall_workflow.compile(checkpointer=checkpointer)
thread_id_4 = uuid.uuid4()
thread_config_4 = {"configurable": {"thread_id": thread_id_4}}

# Run the graph until the first interrupt 
# Email will be classified as "respond" 
# Agent will create a schedule_meeting and write_email tool call
print("Running the graph until the first interrupt...")
for chunk in graph.stream({"email_input": email_input_respond}, config=thread_config_4):
    # Inspect interrupt object if present
    if '__interrupt__' in chunk:
        Interrupt_Object = chunk['__interrupt__'][0]
        print("\nINTERRUPT OBJECT:")
        print(f"Action Request: {Interrupt_Object.value[0]['action_request']}")

Running the graph until the first interrupt...
📧 Classification: RESPOND - This email requires a response

INTERRUPT OBJECT:
Action Request: {'action': 'Question', 'args': {'content': 'What day and time are you considering for the brunch in 3 weeks?'}}


Ignore the `question` tool call

When the agent asks for clarification about the brunch plans, we simulate a user ignoring the question:

1. The agent has asked about preferred day and time for the brunch
2. We provide an "ignore" response to this question
3. The system processes this as a decision to abandon handling this email:
   - A message is added stating "User ignored this question. Ignore this email and end the workflow."
   - The workflow ends without sending any response

This pattern is important because it allows users to:
- Decide that some emails don't actually need responses
- Avoid providing information they're not ready to share
- Defer decision-making to a later time
- Take over email handling manually if they prefer

The trace and message history show how cleanly the workflow ends after ignoring the question.

In [25]:
print(f"\nSimulating user ignoring the {Interrupt_Object.value[0]['action_request']} tool call...")
for chunk in graph.stream(Command(resume=[{"type": "ignore"}]), config=thread_config_4):
    # Inspect interrupt object if present
    if '__interrupt__' in chunk:
        Interrupt_Object = chunk['__interrupt__'][0]
        print("\nINTERRUPT OBJECT:")
        print(f"Action Request: {Interrupt_Object.value[0]['action_request']}")


Simulating user ignoring the {'action': 'Question', 'args': {'content': 'What day and time are you considering for the brunch in 3 weeks?'}} tool call...


As before, we end:

https://smith.langchain.com/public/276c4016-2b4c-43f5-a677-834a5eaa47c0/r

In [26]:
state = graph.get_state(thread_config_4)
for m in state.values['messages']:
    m.pretty_print()

================================ Human Message =================================

Respond to the email: 

**Subject**: Meet Jim and Lisa for brunch in 3 weeks?
**From**: Partner <partner@home.com>
**To**: Lance Martin <lance@company.com>

Hey, should we invite Jim and Lisa to brunch in 3 weeks? We could go to the new place on 17th that everyone is talking about.

---

================================== Ai Message ==================================
Tool Calls:
  Question (call_l6LC1srm8qt8CJCihDTz7wAW)
 Call ID: call_l6LC1srm8qt8CJCihDTz7wAW
  Args:
    content: What day and time are you considering for the brunch in 3 weeks?
================================= Tool Message =================================

User ignored this question. Ignore this email and end the workflow.


## Respond (with feedback) `write_email`, `schedule_meeting`, and `question`

This test set demonstrates the "response" capability - providing feedback without editing or accepting:

1. First, we test feedback for meeting scheduling:
   - The user provides specific preferences (30 minutes instead of 45, and afternoon meetings)
   - The agent incorporates this feedback into a revised proposal
   - The user then accepts the revised meeting schedule

2. Second, we test feedback for email drafting:
   - The user requests a shorter, less formal email with a specific closing statement
   - The agent completely rewrites the email according to this guidance
   - The user accepts the new draft

3. Lastly, we test feedback for questions:
   - For the brunch invitation, the user answers the question with additional context
   - The agent uses this information to draft an appropriate email response
   - The workflow proceeds with the user's input integrated

The "response" capability bridges the gap between acceptance and editing - users can guide the agent without having to write the full content themselves. This is especially powerful for:
- Adjusting tone and style
- Adding context the agent missed
- Redirecting the agent's approach
- Answering questions in a way that shapes the next steps

In [27]:
# Respond - Meeting Request Email
email_input_respond = {
    "to": "Lance Martin <lance@company.com>",
    "author": "Project Manager <pm@client.com>",
    "subject": "Tax season let's schedule call",
    "email_thread": "Lance,\n\nIt's tax season again, and I wanted to schedule a call to discuss your tax planning strategies for this year. I have some suggestions that could potentially save you money.\n\nAre you available sometime next week? Tuesday or Thursday afternoon would work best for me, for about 45 minutes.\n\nRegards,\nProject Manager"
}

# Compile the graph
checkpointer = MemorySaver()
graph = overall_workflow.compile(checkpointer=checkpointer)
thread_id_5 = uuid.uuid4()
thread_config_5 = {"configurable": {"thread_id": thread_id_5}}

# Run the graph until the first interrupt 
# Email will be classified as "respond" 
# Agent will create a schedule_meeting and write_email tool call
print("Running the graph until the first interrupt...")
for chunk in graph.stream({"email_input": email_input_respond}, config=thread_config_5):
    # Inspect interrupt object if present
    if '__interrupt__' in chunk:
        Interrupt_Object = chunk['__interrupt__'][0]
        print("\nINTERRUPT OBJECT:")
        print(f"Action Request: {Interrupt_Object.value[0]['action_request']}")

Running the graph until the first interrupt...
📧 Classification: RESPOND - This email requires a response

INTERRUPT OBJECT:
Action Request: {'action': 'schedule_meeting', 'args': {'attendees': ['pm@client.com', 'lance@company.com'], 'subject': 'Tax Planning Strategies Discussion', 'duration_minutes': 45, 'preferred_day': '2025-04-22', 'start_time': 14}}


Provide feedback for the `schedule_meeting` tool call

Now we explore the feedback capability for meeting scheduling:

1. The agent proposes the standard 45-minute meeting on Tuesday at 2:00 PM
2. Instead of accepting or editing, we provide feedback in natural language
3. Our feedback specifies two preferences:
   - Shorter meeting (30 minutes instead of 45)
   - Preference for afternoon meetings (after 2pm)
4. The agent receives this feedback through the `response` type
5. The interrupt handler adds this feedback as a message to the state
6. The agent processes this feedback and generates a new tool call incorporating these preferences

Unlike direct editing, which requires specifying the entire set of parameters, feedback allows users to express their preferences conversationally. The agent must then interpret this feedback and apply it appropriately to create a revised proposal.

In [28]:
print(f"\nSimulating user providing feedback for the {Interrupt_Object.value[0]['action_request']['action']} tool call...")
for chunk in graph.stream(Command(resume=[{"type": "response", "args": "Please schedule this for 30 minutes instead of 45 minutes, and I prefer afternoon meetings after 2pm."}]), config=thread_config_5):
    # Inspect interrupt object if present
    if '__interrupt__' in chunk:
        Interrupt_Object = chunk['__interrupt__'][0]
        print("\nINTERRUPT OBJECT:")
        print(f"Action Request: {Interrupt_Object.value[0]['action_request']}")


Simulating user providing feedback for the schedule_meeting tool call...

INTERRUPT OBJECT:
Action Request: {'action': 'schedule_meeting', 'args': {'attendees': ['pm@client.com', 'lance@company.com'], 'subject': 'Tax Planning Strategies Discussion', 'duration_minutes': 30, 'preferred_day': '2025-04-22', 'start_time': 14}}


Accept the `schedule_meeting` tool call after providing feedback

In [29]:
print(f"\nSimulating user accepting the {Interrupt_Object.value[0]['action_request']} tool call...")
for chunk in graph.stream(Command(resume=[{"type": "accept"}]), config=thread_config_5):
    # Inspect interrupt object if present
    if '__interrupt__' in chunk:
        Interrupt_Object = chunk['__interrupt__'][0]
        print("\nINTERRUPT OBJECT:")
        print(f"Action Request: {Interrupt_Object.value[0]['action_request']}")


Simulating user accepting the {'action': 'schedule_meeting', 'args': {'attendees': ['pm@client.com', 'lance@company.com'], 'subject': 'Tax Planning Strategies Discussion', 'duration_minutes': 30, 'preferred_day': '2025-04-22', 'start_time': 14}} tool call...

INTERRUPT OBJECT:
Action Request: {'action': 'write_email', 'args': {'to': 'pm@client.com', 'subject': "Re: Tax season let's schedule call", 'content': "Hello,\n\nThank you for reaching out. I've scheduled a call for us to discuss tax planning strategies on Tuesday, April 22, 2025, at 2:00 PM. The meeting is set for 30 minutes.\n\nLooking forward to our discussion.\n\nBest regards,\n\nLance Martin"}}


Now provide feedback for the `write_email` tool call

After accepting the revised meeting schedule, the agent drafts an email. We now test feedback for email content:

1. The agent's email is relatively formal and detailed
2. We provide stylistic feedback requesting:
   - A shorter, more concise email
   - A less formal tone
   - A specific closing statement about looking forward to the meeting
3. The agent processes this feedback to completely rewrite the email
4. The new draft is much shorter, more casual, and includes the requested closing

This demonstrates the power of natural language feedback for content creation:
- Users don't need to rewrite the entire email themselves
- They can provide high-level guidance on style, tone, and content
- The agent handles the actual writing based on this guidance
- The result better matches user preferences while preserving the essential information

The message history shows both the original and revised emails, clearly showing how the feedback was incorporated.

In [30]:
print(f"\nSimulating user providing feedback for the {Interrupt_Object.value[0]['action_request']['action']} tool call...")
for chunk in graph.stream(Command(resume=[{"type": "response", "args": "Shorter and less formal. Include a closing statement about looking forward to the meeting!"}]), config=thread_config_5):
    # Inspect response_agent most recent message
    if 'response_agent' in chunk:
        chunk['response_agent']['messages'][-1].pretty_print()
    # Inspect interrupt object if present
    if '__interrupt__' in chunk:
        Interrupt_Object = chunk['__interrupt__'][0]
        print("\nINTERRUPT OBJECT:")
        print(f"Action Request: {Interrupt_Object.value[0]['action_request']}")


Simulating user providing feedback for the write_email tool call...

INTERRUPT OBJECT:
Action Request: {'action': 'write_email', 'args': {'to': 'pm@client.com', 'subject': "Re: Tax season let's schedule call", 'content': "Hi,\n\nI've set up our call for Tuesday, April 22, at 2:00 PM for 30 minutes. Looking forward to it!\n\nBest,\nLance"}}


Accept the `write_email` tool call after providing feedback

In [31]:
print(f"\nSimulating user accepting the {Interrupt_Object.value[0]['action_request']} tool call...")
for chunk in graph.stream(Command(resume=[{"type": "accept"}]), config=thread_config_5):
    # Inspect interrupt object if present
    if '__interrupt__' in chunk:
        Interrupt_Object = chunk['__interrupt__'][0]
        print("\nINTERRUPT OBJECT:")
        print(f"Action Request: {Interrupt_Object.value[0]['action_request']}")


Simulating user accepting the {'action': 'write_email', 'args': {'to': 'pm@client.com', 'subject': "Re: Tax season let's schedule call", 'content': "Hi,\n\nI've set up our call for Tuesday, April 22, at 2:00 PM for 30 minutes. Looking forward to it!\n\nBest,\nLance"}} tool call...


Look at the full message history, and see the trace:

https://smith.langchain.com/public/57006770-6bb3-4e40-b990-143c373ebe60/r

We can see that user feedback in incorporated into the tool calls.  

In [32]:
state = graph.get_state(thread_config_5)
for m in state.values['messages']:
    m.pretty_print()

================================ Human Message =================================

Respond to the email: 

**Subject**: Tax season let's schedule call
**From**: Project Manager <pm@client.com>
**To**: Lance Martin <lance@company.com>

Lance,

It's tax season again, and I wanted to schedule a call to discuss your tax planning strategies for this year. I have some suggestions that could potentially save you money.

Are you available sometime next week? Tuesday or Thursday afternoon would work best for me, for about 45 minutes.

Regards,
Project Manager

---

================================== Ai Message ==================================
Tool Calls:
  check_calendar_availability (call_5Cs51CxeKicG85x0xeeZgRxg)
 Call ID: call_5Cs51CxeKicG85x0xeeZgRxg
  Args:
    day: 2025-04-22
  check_calendar_availability (call_c7m8uYJaVH9x6l5zJdUj3HMk)
 Call ID: call_c7m8uYJaVH9x6l5zJdUj3HMk
  Args:
    day: 2025-04-24
================================= Tool Message =================================

Ava

Now let's try an email that calls the `Question` tool to provide feedback

Finally, we test how feedback works with the `Question` tool:

1. For the brunch invitation email, the agent asks about preferred day and time
2. Instead of ignoring, we provide a substantive response with additional context:
   - Confirming we want to invite the people mentioned
   - Noting we need to check which weekend works best
   - Adding information about needing a reservation
3. The agent uses this information to:
   - Draft a comprehensive email response incorporating all our feedback
   - Notice we didn't provide a specific day/time, so it suggests checking the calendar
   - Include the detail about making a reservation
4. The complete email reflects both the original request and our additional guidance

This demonstrates how question responses can shape the entire workflow:
- Questions let the agent gather missing information
- User responses can include both direct answers and additional context
- The agent integrates all this information into its next actions
- The final outcome reflects the collaborative intelligence of both human and AI

In [33]:
# Respond - Meeting Request Email
email_input_respond = {
    "to": "Lance Martin <lance@company.com>",
    "author": "Partner <partner@home.com>",
    "subject": "Meet Jim and Lisa for brunch in 3 weeks?",
    "email_thread": "Hey, should we invite Jim and Lisa to brunch in 3 weeks? We could go to the new place on 17th that everyone is talking about."
}

# Compile the graph
checkpointer = MemorySaver()
graph = overall_workflow.compile(checkpointer=checkpointer)
thread_id_6 = uuid.uuid4()
thread_config_6 = {"configurable": {"thread_id": thread_id_6}}

# Run the graph until the first interrupt
print("Running the graph until the first interrupt...")
for chunk in graph.stream({"email_input": email_input_respond}, config=thread_config_6):
    # Inspect interrupt object if present
    if '__interrupt__' in chunk:
        Interrupt_Object = chunk['__interrupt__'][0]
        print("\nINTERRUPT OBJECT:")
        print(f"Action Request: {Interrupt_Object.value[0]['action_request']}")

Running the graph until the first interrupt...
📧 Classification: RESPOND - This email requires a response

INTERRUPT OBJECT:
Action Request: {'action': 'Question', 'args': {'content': 'What day and time are you considering for the brunch in 3 weeks? I can check my calendar for availability.'}}


Provide feedback for the `Question` tool call

In [34]:
print(f"\nSimulating user providing feedback for the {Interrupt_Object.value[0]['action_request']['action']} tool call...")
for chunk in graph.stream(Command(resume=[{"type": "response", "args": "Yes, let's invite them, but let me confirm which weekend works best. Also mention that we'll need to make a reservation since that place is popular."}]), config=thread_config_6):
    # Inspect interrupt object if present
    if '__interrupt__' in chunk:
        Interrupt_Object = chunk['__interrupt__'][0]
        print("\nINTERRUPT OBJECT:")
        print(f"Action Request: {Interrupt_Object.value[0]['action_request']}")


Simulating user providing feedback for the Question tool call...

INTERRUPT OBJECT:
Action Request: {'action': 'write_email', 'args': {'to': 'partner@home.com', 'subject': 'Re: Meet Jim and Lisa for brunch in 3 weeks?', 'content': "Hi,\n\nYes, let's definitely invite Jim and Lisa for brunch. I'll check my calendar to confirm which weekend works best for us. Also, since the new place on 17th is quite popular, we'll need to make a reservation in advance.\n\nI'll get back to you shortly with the best date.\n\nBest,\nLance"}}


Accept the `write_email` tool call

In [35]:
print(f"\nSimulating user accepting the {Interrupt_Object.value[0]['action_request']['action']} tool call...")
for chunk in graph.stream(Command(resume=[{"type": "accept"}]), config=thread_config_6):
    # Inspect response_agent most recent message
    if 'response_agent' in chunk:
        chunk['response_agent']['messages'][-1].pretty_print()
    # Inspect interrupt object if present
    if '__interrupt__' in chunk:
        Interrupt_Object = chunk['__interrupt__'][0]
        print("\nINTERRUPT OBJECT:")
        print(f"Action Request: {Interrupt_Object.value[0]['action_request']}")


Simulating user accepting the write_email tool call...
================================== Ai Message ==================================
Tool Calls:
  Done (call_TWXVmN0lmZpypDo7TZwEEWuw)
 Call ID: call_TWXVmN0lmZpypDo7TZwEEWuw
  Args:
    done: True


Look at the full message history, and see the trace:

https://smith.langchain.com/public/f4c727c3-b1d9-47a5-b3d0-3451619db8a2/r

We can see that user feedback in incorporated into the email response.

In [36]:
state = graph.get_state(thread_config_6)
for m in state.values['messages']:
    m.pretty_print()

================================ Human Message =================================

Respond to the email: 

**Subject**: Meet Jim and Lisa for brunch in 3 weeks?
**From**: Partner <partner@home.com>
**To**: Lance Martin <lance@company.com>

Hey, should we invite Jim and Lisa to brunch in 3 weeks? We could go to the new place on 17th that everyone is talking about.

---

================================== Ai Message ==================================
Tool Calls:
  Question (call_ebOvuGZuyzNxziFBrYtXifeJ)
 Call ID: call_ebOvuGZuyzNxziFBrYtXifeJ
  Args:
    content: What day and time are you considering for the brunch in 3 weeks? I can check my calendar for availability.
================================= Tool Message =================================

User answered the question, which can we can use for any follow up actions. Feedback: Yes, let's invite them, but let me confirm which weekend works best. Also mention that we'll need to make a reservation since that place is popular.
========

## Test Case for Notify Classification

This test demonstrates how the system handles emails classified as "NOTIFY" and how users can respond to notifications:

1. The triage system classifies important informational emails as "NOTIFY" when:
   - They contain important information (like security updates)
   - They don't require immediate action
   - They should be brought to the user's attention
   
2. For notify classifications:
   - The workflow routes to the `triage_interrupt_handler`
   - The user sees the email content with options to ignore or respond
   - No default action is suggested

3. In this test, we:
   - Process an IT security update email that gets classified as "NOTIFY"
   - Simulate the user deciding to respond with specific feedback
   - See how the agent drafts an appropriate response based on this feedback
   - Approve the response to be sent

This showcases how the HITL system can transform a passive notification into an active response when the user decides one is warranted, bridging the gap between the initial "NOTIFY" classification and a full response.

In [37]:
# Notify - Important FYI Email
email_input_notify = {
    "to": "Team Members <team@company.com>",
    "author": "IT Department <it@company.com>",
    "subject": "Critical Security Update",
    "email_thread": "Dear Team,\n\nThis is an important security notification. We will be updating our authentication system this weekend. During the update window (Saturday 2am-4am), you will not be able to access company resources.\n\nPlease ensure you log out of all systems before the maintenance window.\n\nRegards,\nIT Department"
}

# Compile the graph with new thread
checkpointer = MemorySaver()
graph = overall_workflow.compile(checkpointer=checkpointer)
thread_id_7 = uuid.uuid4()
thread_config_7 = {"configurable": {"thread_id": thread_id_7}}

# Run the graph until the first interrupt - should be classified as "notify"
print("Running the graph until the first interrupt...")
for chunk in graph.stream({"email_input": email_input_notify}, config=thread_config_7):
    # Inspect interrupt object if present
    if '__interrupt__' in chunk:
        Interrupt_Object = chunk['__interrupt__'][0]
        print("\nINTERRUPT OBJECT:")
        print(f"Action Request: {Interrupt_Object.value[0]['action_request']}")

Running the graph until the first interrupt...
🔔 Classification: NOTIFY - This email contains important information

INTERRUPT OBJECT:
Action Request: {'action': 'Email Assistant: notify', 'args': {}}


Now simulate user deciding to respond with feedback.

Although the email was classified as "NOTIFY" (meaning it normally wouldn't require a response), the HITL system gives users the flexibility to override this classification. In this step:

1. We provide feedback indicating we want to acknowledge receipt of the security notice
2. The `triage_interrupt_handler` processes this feedback:
   - It adds the user's guidance as a message to the state
   - It routes to the `response_agent` node instead of ending
3. The response agent uses this guidance to draft an appropriate acknowledgment email
4. An interrupt is created for the user to review this draft before sending

This demonstrates an important capability: the ability for users to override the initial classification when they feel a response is warranted. This ensures that the system remains flexible and adaptable to user preferences, while still providing useful initial triage.

In [38]:
print("\nSimulating user deciding to respond with feedback...")
for chunk in graph.stream(Command(resume=[{"type": "response", "args": "We should acknowledge receipt of this important notice and confirm that we'll be logged out before the maintenance window."}]), config=thread_config_7):
    # Inspect interrupt object if present
    if '__interrupt__' in chunk:
        Interrupt_Object = chunk['__interrupt__'][0]
        print("\nINTERRUPT OBJECT:")
        print(f"Action Request: {Interrupt_Object.value[0]['action_request']}")


Simulating user deciding to respond with feedback...

INTERRUPT OBJECT:
Action Request: {'action': 'write_email', 'args': {'to': 'it@company.com', 'subject': 'Re: Critical Security Update', 'content': 'Dear IT Department,\n\nThank you for the important security update notification. I acknowledge receipt of this notice and confirm that I will ensure to log out of all systems before the maintenance window this Saturday from 2am to 4am.\n\nRegards,\nLance'}}


Accept the `write_email` tool call after feedback.

In [39]:
print(f"\nSimulating user accepting the {Interrupt_Object.value[0]['action_request']} tool call...")
for chunk in graph.stream(Command(resume=[{"type": "accept"}]), config=thread_config_7):
    # Inspect interrupt object if present
    if '__interrupt__' in chunk:
        Interrupt_Object = chunk['__interrupt__'][0]
        print("\nINTERRUPT OBJECT:")
        print(f"Action Request: {Interrupt_Object.value[0]['action_request']}")


Simulating user accepting the {'action': 'write_email', 'args': {'to': 'it@company.com', 'subject': 'Re: Critical Security Update', 'content': 'Dear IT Department,\n\nThank you for the important security update notification. I acknowledge receipt of this notice and confirm that I will ensure to log out of all systems before the maintenance window this Saturday from 2am to 4am.\n\nRegards,\nLance'}} tool call...


Look at the full message history, and see the trace:

https://smith.langchain.com/public/6594f98f-eb83-4560-9c34-28ec22ada3dc/r

We can see that user feedback causes agent to go reply to the email.

In [40]:
state = graph.get_state(thread_config_7)
for m in state.values['messages']:
    m.pretty_print()

================================ Human Message =================================

Email to notify user about: 

**Subject**: Critical Security Update
**From**: IT Department <it@company.com>
**To**: Team Members <team@company.com>

Dear Team,

This is an important security notification. We will be updating our authentication system this weekend. During the update window (Saturday 2am-4am), you will not be able to access company resources.

Please ensure you log out of all systems before the maintenance window.

Regards,
IT Department

---

================================ Human Message =================================

User wants to reply to the email. Use this feedback to respond: We should acknowledge receipt of this important notice and confirm that we'll be logged out before the maintenance window.
================================== Ai Message ==================================
Tool Calls:
  write_email (call_9fi3k78DE98MFtey5aYSaql7)
 Call ID: call_9fi3k78DE98MFtey5aYSaql7
  Args

## Test Case for Notify + Ignore

This test demonstrates the other path for notifications - when users choose to simply acknowledge without responding:

1. We process a company picnic announcement email, which gets classified as "NOTIFY"
2. The user decides this notification needs no response and chooses to ignore it
3. The workflow ends immediately with no further action

This scenario highlights several key aspects of the HITL system:
- The initial triage correctly identifies information that's worth seeing but doesn't require action
- Users can quickly process such notifications with minimal interaction
- The system respects the user's decision not to act
- The workflow efficiently ends without wasting time on unnecessary steps

Together with the previous test, this demonstrates the complete notification handling workflow:
- Some notifications warrant responses (previous test)
- Others simply need acknowledgment (this test)
- The user maintains control over which path to take

In [41]:
# Notify - Important FYI Email
email_input_notify = {
    "to": "Team Members <team@company.com>",
    "author": "HR Department <hr@company.com>",
    "subject": "Company Picnic Next Month",
    "email_thread": "Dear Team,\n\nWe're planning the annual company picnic for next month. The tentative date is Saturday, June 15th from noon to 4pm at Central Park. There will be food, games, and activities for families.\n\nMore details will follow in the coming weeks.\n\nRegards,\nHR Department"
}

# Compile the graph with new thread
checkpointer = MemorySaver()
graph = overall_workflow.compile(checkpointer=checkpointer)
thread_id_8 = uuid.uuid4()
thread_config_8 = {"configurable": {"thread_id": thread_id_8}}

# Run the graph until the first interrupt - should be classified as "notify"
print("Running the graph until the first interrupt...")
for chunk in graph.stream({"email_input": email_input_notify}, config=thread_config_8):
    # Inspect interrupt object if present
    if '__interrupt__' in chunk:
        Interrupt_Object = chunk['__interrupt__'][0]
        print("\nINTERRUPT OBJECT:")
        print(f"Action Request: {Interrupt_Object.value[0]['action_request']}")

Running the graph until the first interrupt...
🔔 Classification: NOTIFY - This email contains important information

INTERRUPT OBJECT:
Action Request: {'action': 'Email Assistant: notify', 'args': {}}


Now simulate user deciding to ignore the notification.

In this step, we simulate the simplest path for notification handling - acknowledgment without action:

1. We receive the interrupt with the notification about the company picnic
2. We respond with the "ignore" response type
3. The workflow immediately ends without creating any responses
4. The message history shows only the notification itself, with no additional processing

This straightforward path is actually critical for workflow efficiency:
- It allows users to quickly process informational emails
- It avoids the overhead of unnecessary response generation
- It recognizes that many notifications simply need to be seen, not answered
- It respects the user's time by ending the workflow immediately

The complete message history shows how clean this path is - just the notification itself, with no additional messages once the user chooses to ignore it.

In [42]:
print("\nSimulating user deciding to ignore the notification...")
for chunk in graph.stream(Command(resume=[{"type": "ignore"}]), config=thread_config_8):
    # Inspect interrupt object if present
    if '__interrupt__' in chunk:
        Interrupt_Object = chunk['__interrupt__'][0]
        print("\nINTERRUPT OBJECT:")
        print(f"Action Request: {Interrupt_Object.value[0]['action_request']}")


Simulating user deciding to ignore the notification...


Look at the full message history, and see the trace:

https://smith.langchain.com/public/8193f616-244f-471d-8ec6-bd39624a0c88/r

Here, we can see that if we ignore the notification, we just end. 

In [43]:
state = graph.get_state(thread_config_8)
for m in state.values['messages']:
    m.pretty_print()

================================ Human Message =================================

Email to notify user about: 

**Subject**: Company Picnic Next Month
**From**: HR Department <hr@company.com>
**To**: Team Members <team@company.com>

Dear Team,

We're planning the annual company picnic for next month. The tentative date is Saturday, June 15th from noon to 4pm at Central Park. There will be food, games, and activities for families.

More details will follow in the coming weeks.

Regards,
HR Department

---

